1. Documentati-va despre [dashboards](https://en.wikipedia.org/wiki/Dashboard_(business)) (~ "tablouri de bord"). 
1. Documentati-va despre solutiile de dashboards disponibile in jupyter notebook, de ex:
    1. [How to create a dashboard in Python with Jupyter Notebook?](https://mljar.com/blog/dashboard-python-jupyter-notebook/)
    1. [Interactive Dashboard from Jupyter Notebook with Mercury framework](https://towardsdatascience.com/interactive-dashboard-from-jupyter-notebook-with-mercury-framework-e1269fdbe73c)
    1. [How to Create a Beautiful Python Visualization Dashboard With Panel/Hvplot](https://www.youtube.com/watch?v=uhxiXOTKzfs)
    1. [Interactive data dashboards in Jupyter notebook with ipywidgets and Bokeh](https://danielmuellerkomorowska.com/2021/08/02/interactive-data-dashboards-in-jupyter-notebook-with-ipywidgets-and-bokeh/); explicatii la [How to Build an Interactive Data Dashboard in Jupyter Notebook](https://www.youtube.com/watch?v=EEKX5l_Y3_8) etc.
1. Alegeti un set de date din repository-ul [UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) si realizati un dashboard pe baza lui. Prezentarea se poate face in Jupyter notebook sau ca aplicatie standalone. Includeti in arhiva:
    * fisier `requirements.txt` cu bibliotecile care trebuie instalate cu `pip`; [documentatie requirements.txt](https://learnpython.com/blog/python-requirements-file/)
    * fisier readme.txt, in limba romana sau engleza, cu pasii care trebuie urmati pentru a putea utiliza dashboard-ul; de exemplu, unul din pasi este `pip install -r requirements.txt`; adaugati alti pasi necesari. 

In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [2]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():
    
    header_list = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 
                  'marital-status', 'occupation', 'relationship', 'race', 'sex',
                  'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
    
    df = pd.read_csv('adult.data', header=None, names = header_list, index_col=False)

    pn.state.cache['data'] = df.copy()

else: 

    df = pn.state.cache['data']

In [3]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# Make DataFrame Pipeline Interactive
idf = df.interactive()

In [5]:
age_slider = pn.widgets.IntSlider(name='Age slider', start=df['age'].min(), end=df['age'].max(), step=1, value=22)
age_slider

IntSlider(end=90, name='Age slider', start=17, value=22)

In [6]:
yaxis_capital = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['capital-gain', 'capital-loss',],
    button_type='success'
)

In [7]:
s = [' Male', ' Female']

capital_pipeline = (
    idf[
        (idf.age <= age_slider) &
        (idf.sex.isin(s))
    ]
    .groupby(['sex', 'age'])[yaxis_capital].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='age')  
    .reset_index(drop=True)
)

In [8]:
capital_pipeline.head()

In [9]:
capital_plot = capital_pipeline.hvplot(x = 'age',  by='sex', y=yaxis_capital, line_width=2, title="")
capital_plot

In [10]:
capital_table = capital_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
capital_table

In [11]:
income_greater_pipeline = (
    idf[
        (idf.age <= age_slider) &
        (idf.income == ' >50K')
    ]
    .groupby(['age', 'sex', 'education', 'occupation', 'income'])[yaxis_capital].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='age')  
    .reset_index(drop=True)
)

In [12]:
income_greater_pipeline.head()

In [13]:
income_greater_table = income_greater_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
income_greater_table

In [14]:
income_less_pipeline = (
    idf[
        (idf.age <= age_slider) &
        (idf.income == ' <=50K')
    ]
    .groupby(['age', 'sex', 'education', 'occupation'])[yaxis_capital].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='age')  
    .reset_index(drop=True)
)

In [15]:
income_less_table = income_less_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
income_less_table

In [16]:
template = pn.template.FastListTemplate(
    title='Adul data set dashboard', 
    sidebar=[pn.pane.Markdown("# Adult Data Set"), 
             pn.pane.Markdown("## Settings"),   
             age_slider],
    main=[pn.Row(pn.Column(yaxis_capital, 
                           capital_plot.panel(width=700), margin=(0,25))), 
          pn.Row(pn.Column('# Income <=50K', income_less_table.panel(width=560), margin=(0,25)), 
                 pn.Column('# Income >50K',income_greater_table.panel(width=650)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
template.show()
# template.servable();

Launching server at http://localhost:53863


In [17]:
# pip list --format=freeze > requirements.txt